## Data Consolidation Pipeline

## Final Data Structure
We are analyzing horse racing, in our case we want the final data to be every attempt of every horse in every race.
Therefore we'll first fetch all the race results that took place in the last 15 years (11k). For each race, there will be 8-13 horses.
Take their results, combine with that race's background information (what's the course like, wet or dry, length of race..)
Finally for each data point we would also like to know the horse's stats. Therefore we'll have to crawl for the performance of the horse as well.
Finally combining them all together into a super dataset

### Define Length of data

In [ ]:
from scrapers import fetch_races_from_days_ago


# fetch all races for the last 15 years
fetch_races_from_days_ago(15 * 365, 30)

### Fetch Horse data for every Horse that is mentioned in each race

In [ ]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from loaders.consolidate_horse import append_horses_info

directory = "results/"
output_file = f"{directory}/horse_res_15.csv"
files = os.listdir(directory)

existing_links = set()

if os.path.exists(output_file):
    output_df = pd.read_csv(output_file)
    if len(output_df.columns) > 0:
        existing_links = set(output_df.iloc[:, 0])

def process_link(link):
    if pd.notna(link):
        if link not in existing_links:
            print(f"Appending link: {link} to {output_file}")
            append_horses_info(link, output_file)
            return link
        else:
            print(f"Link {link} already exists in {output_file}")
    else:
        print("Empty link")
    return None

# Collect all links to process
links_to_process = []
for file in files:
    if file.endswith(".csv") and not file.endswith("_bg.csv"):
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)

        if len(df.columns) >= 13:
            links_to_process.extend(df.iloc[:, 12].dropna().unique())

# Filter out existing links
links_to_process = [link for link in links_to_process if link not in existing_links]

# Use ThreadPoolExecutor for multithreading
with ThreadPoolExecutor(max_workers=20) as executor:
    futures = {executor.submit(process_link, link): link for link in links_to_process}

    for future in as_completed(futures):
        result = future.result()
        if result:
            existing_links.add(result)

### Combine the background data to the race data
in each race there are 8-13 horses, each of these horses in the end will be a training data point in our model construction process
therefore each of the data point would contain the background information about the race

In [ ]:
from loaders.consolidate_race import append_race_res
import os
import fnmatch
from tqdm import tqdm

directory = "results/"
all_files = os.listdir(directory)
output_file = f"{directory}final2.csv"

csv_files = [os.path.join(directory, f) for f in all_files 
             if fnmatch.fnmatch(f, 'res_*.csv') and not fnmatch.fnmatch(f, '*_bg.csv')]

for file in tqdm(csv_files, desc="Processing files"):
    append_race_res(file, output_file)

## Combine Horse Data with Race Data

In [ ]:
import pandas as pd

directory = "results/"
race_data_file = f"{directory}final2.csv"
horse_data_file = f"{directory}horse_res.csv"

df1 = pd.read_csv(race_data_file)
df2 = pd.read_csv(horse_data_file)

df_merged = pd.merge(df1, df2, on='URL', how='left')

df_merged.to_csv("final_horse.csv")
